<a href="https://colab.research.google.com/github/sanjanagarai/InterIIT_Team1_Task2/blob/main/LXMert_VQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Libraries



In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.2 MB/s eta 0:00:00


In [ ]:
!pip install torch transformers

## Importing modules

In [ ]:
import pandas as pd
from transformers import AutoConfig, AutoTokenizer, AutoModel
import torch
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow.keras import  layers, models,Sequential
from tensorflow import keras
import zipfile

from tensorflow.keras import  layers, models,Sequential
from tensorflow import keras
import tensorflow as tf
from transformers import LxmertTokenizer, LxmertForQuestionAnswering
from tensorflow.keras.layers import Input, Masking, Concatenate, Dense
from tensorflow.keras.models import Model

import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.autograd import Variable
from PIL import Image

Connecting to drive

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


## IMAGE FEATURE EXTRACTION

In [ ]:
# Read the CSV file
existing_df = pd.read_csv('/content/question_data.csv')  # Replace 'your_dataset.csv' with the actual filename

# Select the required columns: Figure_path, question, and answer
new_df = existing_df[['image_name', 'question', 'answer']]

# Save the updated dataset back to q&a_space.csv
new_df.to_csv('q&a_space.csv', index=False)

Loading the RadBERT model

In [ ]:
# loading the model
config = AutoConfig.from_pretrained('zzxslp/RadBERT-RoBERTa-4m')
tokenizer = AutoTokenizer.from_pretrained('zzxslp/RadBERT-RoBERTa-4m')
model = AutoModel.from_pretrained('zzxslp/RadBERT-RoBERTa-4m', config=config)

Some weights of RobertaModel were not initialized from the model checkpoint at zzxslp/RadBERT-RoBERTa-4m and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Performing tokenization of question and answer pairs


In [ ]:
# Read the CSV file containing figure_path, question, and answer
df = new_df  # Replace 'q&a_space.csv' with your actual CSV file

# Tokenize questions and answers and obtain question and answer features
question_features = []
answer_features = []

for question_text, answer_text in zip(df['question'], df['answer']):
    question_text = str(question_text)
    answer_text = str(answer_text)

    # Tokenize question and answer separately
    question_tokens = tokenizer.encode_plus(question_text, add_special_tokens=True, return_tensors='pt')
    answer_tokens = tokenizer.encode_plus(answer_text, add_special_tokens=True, return_tensors='pt')

    # Obtain hidden states for the [CLS] tokens of questions and answers
    with torch.no_grad():
        question_outputs = model(**question_tokens)
        answer_outputs = model(**answer_tokens)

    # Extract the [CLS] token hidden states for questions and answers
    question_feature = question_outputs.last_hidden_state[:, 0, :]
    answer_feature = answer_outputs.last_hidden_state[:, 0, :]

    question_features.append(question_feature)
    answer_features.append(answer_feature)

# Concatenate question and answer features along the second dimension
question_features_tensor = torch.cat(question_features, dim=0)
answer_features_tensor = torch.cat(answer_features, dim=0)

# Concatenate question and answer features to create text features tensor
text_features_tensor = torch.cat((question_features_tensor, answer_features_tensor), dim=1)


Creating attention mask tensor

In [ ]:
attention_mask = (question_features_tensor != 0).float()

In [ ]:
print(attention_mask.shape)
print(question_features_tensor.shape)

torch.Size([2248, 768])
torch.Size([2248, 768])


We have got the combined question and answer feature tensors

---



## IMAGE FEATURE EXTRACTION

We are extracting the features from the image using ResNet152

In [ ]:

# Define the path to the folder containing images
folder_path = '/content/drive/MyDrive/VQA_RAD_Dataset/Images'

# Get a list of all files in the folder
image_files = [file for file in os.listdir(folder_path) if file.lower().endswith(('png', 'jpg', 'jpeg'))]

# Load pre-trained ResNet-152 model
resnet152 = models.resnet152(pretrained=True)
resnet152 = nn.Sequential(*list(resnet152.children())[:-1])  # Remove the last fully connected layer
resnet152.eval()

# Function to extract features from an image
def extract_features(image_path):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension
    input_variable = Variable(input_batch)

    # Extract features using ResNet-152
    with torch.no_grad():
        features = resnet152(input_variable)

    return features.view(-1, 2048).numpy()  # Reshape features to 1x2048 numpy array

# Create a dictionary to store image names and corresponding feature tensors
image_features_dict = {}

# Extract features from all images in the folder and store in the dictionary
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    image_name = os.path.splitext(image_file)[0]  # Get the name of the image without extension
    features = extract_features(image_path)
    image_features_dict[image_name] = features

# Print the dictionary to see the results
print(image_features_dict)


{'synpic58902': array([[0.0031301 , 1.397012  , 1.3849804 , ..., 0.02696916, 0.0755067 ,
        0.05632993]], dtype=float32), 'synpic60423': array([[1.1923105 , 0.8451253 , 0.33828607, ..., 0.18674414, 0.8881889 ,
        0.30791453]], dtype=float32), 'synpic100176': array([[1.6155833 , 1.0648547 , 0.1505315 , ..., 0.22807993, 0.95593566,
        0.3661528 ]], dtype=float32), 'synpic100132': array([[1.1596252 , 1.1542286 , 0.04265462, ..., 0.37462103, 0.57533634,
        0.5687916 ]], dtype=float32), 'synpic59935': array([[0.07499509, 0.57090706, 0.94750136, ..., 0.3554422 , 0.35605225,
        0.29538897]], dtype=float32), 'synpic60254': array([[0.07582521, 1.1600959 , 2.1245024 , ..., 0.14023662, 0.4461109 ,
        0.03410044]], dtype=float32), 'synpic60831': array([[0.03829674, 0.94704837, 0.18255877, ..., 0.10014411, 0.1959626 ,
        0.06064688]], dtype=float32), 'synpic58261': array([[0.9011912 , 0.77026707, 0.82239985, ..., 0.16867091, 0.7993545 ,
        0.19545674]], dtype

In [ ]:
df=pd.read_csv("question_data.csv")
image_id=df['image_name']

In [ ]:
# Create an empty list to store the corresponding image tensors in the same order as questions
ordered_image_tensors = []

# Iterate through the image ID list and append the respective image tensors
for image_name in image_id:
    ordered_image_tensors.append(image_features_dict[image_name])

In [ ]:
print(len(ordered_image_tensors))
print(type(ordered_image_tensors))

2248
<class 'list'>


In [ ]:
image_features=[]
for m in ordered_image_tensors:
  image_features.append(m[0])

In [ ]:
print(len(image_features))

2248


image_features_tensor is the final tensor having the image features

In [ ]:
# Convert the list to a PyTorch tensor
image_features_tensor = torch.tensor(image_features, dtype=torch.float32)

In [ ]:
print(len(image_features_tensor))
print(type(image_features_tensor))

2248
<class 'torch.Tensor'>


In [ ]:
print(image_features_tensor.shape)

torch.Size([2248, 2048])


creating the positional encoding tensor

In [ ]:
def positional_encoder(feature_dim, max_positional_encoding):
    position = np.arange(0, max_positional_encoding)[:, np.newaxis]
    div_term = np.exp(np.arange(0, feature_dim, 2) * -(np.log(10000.0) / feature_dim))
    positional_encoding = np.zeros((max_positional_encoding, feature_dim))
    positional_encoding[:, 0::2] = np.sin(position * div_term)
    positional_encoding[:, 1::2] = np.cos(position * div_term)
    return torch.tensor(positional_encoding, dtype=torch.float32)

# Assuming feature_dim is 2048 and max_positional_encoding is 2248
feature_dim = 2048
max_positional_encoding = 2248

# Generate positional encodings
positional_encoding = positional_encoder(feature_dim, max_positional_encoding)

print(type(positional_encoding))


<class 'torch.Tensor'>


In [ ]:
print(positional_encoding.shape)

torch.Size([2248, 2048])


## Creating the fusion layer

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import LxmertForQuestionAnswering, LxmertTokenizer, AdamW

# Define your LXMERT model and tokenizer
lxmert_model = LxmertForQuestionAnswering.from_pretrained("unc-nlp/lxmert-base-uncased")
tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")

In [ ]:
input_ids = question_features_tensor
visual_feats = image_features_tensor
visual_pos = positional_encoding

print(input_ids.shape)
print(visual_feats.shape)
print(visual_pos.shape)
print(attention_mask.shape)
print(answer_features_tensor.shape)

torch.Size([2248, 768])
torch.Size([2248, 2048])
torch.Size([2248, 2048])
torch.Size([2248, 768])
torch.Size([2248, 768])


In [ ]:
dataset=TensorDataset(input_ids, visual_feats, visual_pos, attention_mask)
#where:
#input_ids = text_features_tensor
#visual_feats = image_features_tensor
#visual_pos = positional_encodings
#attention_mask = attention_mask

NameError: ignored

Training Loop

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import LxmertForQuestionAnswering, LxmertTokenizer, AdamW

# Set the device for GPU training
device = torch.device("cuda:0")

# Define the LXMERT model and tokenizer
lxmert_model = LxmertForQuestionAnswering.from_pretrained("unc-nlp/lxmert-base-uncased")
tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")

# Move the model to the GPU
lxmert_model.to(device)

# Define other parameters
batch_size = 8
accumulation_steps = 2  # Accumulate gradients over 2 batches
num_epochs = 3

# Create a DataLoader for your dataset (replace 'dataset' with your actual dataset)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define the optimizer
optimizer = AdamW(lxmert_model.parameters(), lr=1e-5)

# Set the model to training mode
lxmert_model.train()

# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    accumulated_loss = 0.0

    for i, batch in enumerate(data_loader):
        text_features, image_features, positional_encoding, attention_mask, answer_features = batch

        # Move tensors to the GPU and set data types
        text_features, image_features, positional_encoding, attention_mask, answer_features = (
            text_features.to(torch.long).to(device),
            image_features.to(torch.float).to(device),
            positional_encoding.to(torch.float).to(device),
            attention_mask.to(torch.float).to(device),
            answer_features.to(torch.long).to(device)
        )

        # Ensure that image_features and text_features match the LXMERT model's expectations
        # Reshape image_features to match the model's input
        image_features = image_features.view(image_features.size(0), -1)

        # Forward pass through the LXMERT model
        outputs = lxmert_model(
            input_ids=text_features,
            visual_feats=image_features,
            visual_pos=positional_encoding,
            attention_mask=attention_mask,
        )

        # Ensure answer_features is of data type torch.long
        answer_targets = answer_features.argmax(dim=1).to(torch.long)

        # Calculate the loss using cross-entropy
        loss = torch.nn.functional.cross_entropy(outputs.logits, answer_targets)
        loss = loss / accumulation_steps  # Scale the loss

        # Accumulate gradients
        loss.backward()

        # Update accumulated loss
        accumulated_loss += loss.item()

        if (i + 1) % accumulation_steps == 0 or i == len(data_loader) - 1:
            # Update weights and reset gradients after accumulation_steps batches
            optimizer.step()
            optimizer.zero_grad()

            # Update total loss and print average loss
            total_loss += accumulated_loss
            accumulated_loss = 0.0

    average_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}")

# Save your trained model if needed
lxmert_model.save_pretrained("trained_lxmert_model")
tokenizer.save_pretrained("trained_lxmert_model")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


RuntimeError: ignored

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import LxmertForQuestionAnswering, LxmertTokenizer, AdamW

# Set the device for GPU training
device = torch.device("cuda:0")

# Define the LXMERT model and tokenizer
lxmert_model = LxmertForQuestionAnswering.from_pretrained("unc-nlp/lxmert-base-uncased")
tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")

# Move the model to the GPU
lxmert_model.to(device)

# Define other parameters
batch_size = 8
accumulation_steps = 2  # Accumulate gradients over 2 batches
num_epochs = 3

# Create a DataLoader for your dataset (replace 'dataset' with your actual dataset)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define the optimizer
optimizer = AdamW(lxmert_model.parameters(), lr=1e-5)

# Set the model to training mode
lxmert_model.train()

# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    accumulated_loss = 0.0

    for i, batch in enumerate(data_loader):
        text_features, image_features, positional_encoding, attention_mask, answer_features = batch

        # Move tensors to the GPU and set data types
        text_features, image_features, positional_encoding, attention_mask, answer_features = (
            text_features.to(torch.long).to(device),
            image_features.to(torch.float).to(device),
            positional_encoding.to(torch.float).to(device),
            attention_mask.to(torch.float).to(device),
            answer_features.to(torch.long).to(device)
        )

        # Forward pass through the LXMERT model
        outputs = lxmert_model(
            input_ids=text_features,
            visual_feats=image_features,
            visual_pos=positional_encoding,
            attention_mask=attention_mask,
        )

        # Ensure answer_features is of data type torch.long
        answer_targets = answer_features.argmax(dim=1).to(torch.long)

        # Calculate the loss using cross-entropy
        loss = torch.nn.functional.cross_entropy(outputs.logits, answer_targets)
        loss = loss / accumulation_steps  # Scale the loss

        # Accumulate gradients
        loss.backward()

        # Update accumulated loss
        accumulated_loss += loss.item()

        if (i + 1) % accumulation_steps == 0 or i == len(data_loader) - 1:
            # Update weights and reset gradients after accumulation_steps batches
            optimizer.step()
            optimizer.zero_grad()

            # Update total loss and print average loss
            total_loss += accumulated_loss
            accumulated_loss = 0.0

    average_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}")

# Save your trained model if needed
lxmert_model.save_pretrained("trained_lxmert_model")
tokenizer.save_pretrained("trained_lxmert_model")

In [ ]:
dataset=TensorDataset(input_ids, visual_feats, visual_pos, attention_mask)
#where:
#input_ids = text_features_tensor
#visual_feats = image_features_tensor
#visual_pos = positional_encodings
#attention_mask = attention_mask